# Best Fit Plane

The idea is to get the points representing the holes of a ring and find the best fit plane.
This document walks through the theory and the math.


# References

- https://www.ilikebigbits.com/2015_03_04_plane_from_points.html <- this is a really good break down of the basics involved and includes source code for the basics
- https://riptutorial.com/numpy/example/16034/find-the-least-squares-solution-to-a-linear-system-with-np-linalg-lstsq
- https://gist.github.com/amroamroamro/1db8d69b4b65e8bc66a6
- https://stats.stackexchange.com/questions/326239/fitted-planes-normal-vector-not-perpendicular
- http://www.songho.ca/math/plane/plane.html
- https://www.ilikebigbits.com/2015_03_04_plane_from_points.html

Best Fit Discussions:

- https://math.stackexchange.com/questions/99299/best-fitting-plane-given-a-set-of-points
- https://www.ilikebigbits.com/2015_03_04_plane_from_points.html


All of these examples assume an XY plane will fit the data. That isn't always the case! Some data may be mostly vertically polarized:

- https://riptutorial.com/numpy/example/16034/find-the-least-squares-solution-to-a-linear-system-with-np-linalg-lstsq
- https://gist.github.com/amroamroamro/1db8d69b4b65e8bc66a6
- https://stats.stackexchange.com/questions/326239/fitted-planes-normal-vector-not-perpendicular


## Plane Equation

The theory will be developed to fit a plane to a set of 3D points in this section. Recall that a plane is described by a normal vector $(\vec{n})$ and a vector on the plane, $(\vec{p})$.

$$\vec{n} = \left \langle a,b,c \right \rangle$$

$$\vec{p} = \vec{p} - \vec{p_1}$$

$$\vec{p} = \left \langle x,y,z \right \rangle$$

$$\vec{p_1} = \left \langle x_1,y_1,z_1 \right \rangle$$

Therefore, the equation of the plane is:

$$\vec{n} \cdot \vec{p} = 0 $$

Simplifying:

$$\left(a, b, c \right) \cdot \left( x - x_1, y - y_1, z - z_1, \right) = 0$$

$$ a \left( x - x_1 \right) + b \left( y - y_1 \right) + c \left( z - z_1 \right) = 0 $$

$$ ax + by + cz - \left( ax + by + cz \right) = 0$$



Let $d = - \left( ax + by + cz \right)$ the plane equation takes it standard form:

$$ ax + by + cz +d = 0$$

> NOTE: If the normal vector, $(\vec{n})$, is normalized, $d$ becomes the distance from the origin of the coordinate system, $\left( 0,0,0 \right)$, to the plane at point $\vec{p_1}$. This also makes $\vec{p_1}$ the plane origin.


## Best Fit Plane for n-3D points

We can use the plane equation in a least squares approach to solve this problem. However, the system is overdetermined, that is the solution space is a 3-dimensional plane and the plane equation has 4 variables. To proceed we need to remove one component of the normal vector, $\vec{n} = \left \langle a,b,c \right \rangle$. We do this by arbitrarily setting one of the components equals to 1.  

We must choose which component to set to 1, based on the best plane to fit the point cloud, XY, XZ, YZ. To decide which plane to take, simply take the determinant of $ A^T A$.

$$ \text{det} \left | A^T A \right |$$

The largest determinant is the plane to select to fit the points.

### XY Solution Plane

If we set $c = 1$ the plane equation becomes:

$$ ax + by + z +d = 0$$

$$ ax + by + d = - z$$

We need to solve for $a,b,d$ and the most efficient way to so construct the matrix system $Ax = b$

$$
A = \begin{bmatrix}
x_0    & y_0    & 1 \\ 
x_1    & y_1    & 1 \\ 
\vdots & \vdots & \vdots \\ 
x_n    & y_n    & 1 
\end{bmatrix}
$$

$$
x = \begin{bmatrix}
a\\ 
b\\ 
d
\end{bmatrix}
$$

$$
b = \begin{bmatrix}
-z_0\\ 
-z_1\\ 
\vdots\\ 
-z_n
\end{bmatrix}
$$

The equation assembled:

$$
\begin{bmatrix}
x_0    & y_0    & 1 \\ 
x_1    & y_1    & 1 \\ 
\vdots & \vdots & \vdots \\ 
x_n    & y_n    & 1 
\end{bmatrix}
\begin{bmatrix}
a\\ 
b\\ 
d
\end{bmatrix} =
\begin{bmatrix}
-z_0\\ 
-z_1\\ 
\vdots\\ 
-z_n
\end{bmatrix}
$$

Multiply by $A^T$:

$$
\begin{bmatrix}
x_0 & x_1 & ... & x_n \\ 
y_0 & y_1 & ... & y_n \\ 
1   & 1   & ... & 1
\end{bmatrix}
\begin{bmatrix}
x_0    & y_0    & 1 \\ 
x_1    & y_1    & 1 \\ 
\vdots & \vdots & \vdots \\ 
x_n    & y_n    & 1 
\end{bmatrix}
\begin{bmatrix}
a\\ 
b\\ 
d
\end{bmatrix} =
\begin{bmatrix}
x_0 & x_1 & ... & x_n \\ 
y_0 & y_1 & ... & y_n \\ 
1   & 1   & ... & 1
\end{bmatrix}
\begin{bmatrix}
-z_0\\ 
-z_1\\ 
\vdots\\ 
-z_n
\end{bmatrix}
$$

$$
\begin{bmatrix}
\Sigma x_i x_i & \Sigma x_i y_i & \Sigma x_i \\ 
\Sigma y_i x_i & \Sigma y_i y_i & \Sigma y_i \\ 
\Sigma x_i     & \Sigma y_i     & N
\end{bmatrix}
\begin{bmatrix}
a\\ 
b\\ 
d
\end{bmatrix} =
-1 \cdot 
\begin{bmatrix}
\Sigma x_i z_i \\ 
\Sigma y_i z_i \\ 
0
\end{bmatrix}
$$


Where $N$ is the number of points.

Let us define $x,y,z$ so they are relative to the centroid of the point cloud.

$$\Sigma x = \Sigma y = \Sigma z = 0 $$


$$
\begin{bmatrix}
\Sigma x_i x_i & \Sigma x_i y_i & 0 \\ 
\Sigma y_i x_i & \Sigma y_i y_i & 0 \\ 
0              & 0              & N
\end{bmatrix}
\begin{bmatrix}
a\\ 
b\\ 
d
\end{bmatrix} =
-1 \cdot 
\begin{bmatrix}
\Sigma x_i z_i \\ 
\Sigma y_i z_i \\ 
0
\end{bmatrix}
$$

It can be seen that $N \cdot d = 0$ from the last row of the matrix. This means that $d = 0$

$$
\begin{bmatrix}
\Sigma x_i x_i & \Sigma x_i y_i & 0 \\ 
\Sigma y_i x_i & \Sigma y_i y_i & 0
\end{bmatrix}
\begin{bmatrix}
a\\ 
b
\end{bmatrix} =
-1 \cdot 
\begin{bmatrix}
\Sigma x_i z_i \\ 
\Sigma y_i z_i 
\end{bmatrix}
$$

### XZ Solution Plane

If we set $b = 1$ the plane equation becomes:

$$ ax + y + cz +d = 0$$

$$ ax + cz + d = - y$$

The matrices become

$$
A = \begin{bmatrix}
x_0    & z_0    & 1 \\ 
x_1    & z_1    & 1 \\ 
\vdots & \vdots & \vdots \\ 
x_n    & z_n    & 1 
\end{bmatrix}
$$

$$
x = \begin{bmatrix}
a\\ 
c\\ 
d
\end{bmatrix}
$$

$$
b = \begin{bmatrix}
-y_0\\ 
-y_1\\ 
\vdots\\ 
-y_n
\end{bmatrix}
$$

### YZ Solution Plane

If we set $a = 1$ the plane equation becomes:

$$ x + by + cz +d = 0$$

$$ by + cz + d = -x$$

The matrices become

$$
A = \begin{bmatrix}
y_0    & z_0    & 1 \\ 
y_1    & z_1    & 1 \\ 
\vdots & \vdots & \vdots \\ 
y_n    & z_n    & 1 
\end{bmatrix}
$$

$$
x = \begin{bmatrix}
b\\ 
c\\ 
d
\end{bmatrix}
$$

$$
b = \begin{bmatrix}
-x_0\\ 
-x_1\\ 
\vdots\\ 
-x_n
\end{bmatrix}
$$

## Solution

Given the matrix equation: $Ax = b$ the solution involves:

$$ A^T A x = A^T b $$



----

The difference in the \(d\) value between the **projection-based determinant approach** and the **SVD method** arises from the way each method defines the best-fit plane and handles the data. Let’s break it down:

---

### 1. **Projection-Based Determinant Method**:
- **What it does**:
  - This method computes least squares regressions for projections onto the XY, XZ, and YZ planes.
  - The plane with the largest determinant (\(\text{det}(A^T A)\)) is selected as the best projection for regression.
  - After selecting the projection, it solves the regression problem \( A \cdot x = b \) to find the coefficients \(a, b, c, d\), then normalizes \(a, b, c\).
  - The \(d\) value is derived from the regression.

- **How it fits the plane**:
  - It is influenced by the projection selected (e.g., XY, XZ, or YZ).
  - The resulting plane minimizes errors in that particular projection (e.g., minimizing errors in the \(z\)-coordinate when using the XY projection).
  - This causes a bias in the computation, as it prioritizes minimizing errors along one specific axis.

- **Impact on \(d\)**:
  - The \(d\) value is not a purely geometric property but is computed relative to the chosen projection. This can lead to a discrepancy when compared to more global fitting methods.

---

### 2. **SVD Method**:
- **What it does**:
  - SVD computes the plane that minimizes the orthogonal distances of all points to the plane in 3D space.
  - It treats all dimensions (X, Y, Z) equally without prioritizing any specific axis or projection.
  - The normal vector is derived from the smallest singular value, which corresponds to the direction of least variation in the point cloud.

- **How it fits the plane**:
  - This method ensures that the plane minimizes the total orthogonal distance from all points to the plane in a least-squares sense.
  - The \(d\) value is computed based on the centroid of the points and the normal vector:
    \[
    d = -(\mathbf{n} \cdot \text{centroid})
    \]

- **Impact on \(d\)**:
  - The \(d\) value in this method represents the offset of the plane from the origin when the normal vector is normalized.
  - It is inherently a global property of the plane and is unaffected by any single projection, leading to differences compared to the projection-based method.

---

### Why the Difference in \(d\) Occurs:
1. **Projection Bias in Determinant Method**:
   - The projection-based approach introduces bias by solving the problem in 2D (e.g., XY plane), which minimizes errors along one axis but ignores orthogonal errors.
   - The \(d\) value in this method depends on the chosen projection and the axis-specific regression.

2. **Orthogonality in SVD Method**:
   - The SVD method works in 3D and minimizes the overall orthogonal distances from points to the plane.
   - This makes the \(d\) value a globally optimized parameter, leading to different results from the determinant method.

3. **Numerical Differences**:
   - Even when normalized, numerical approximations (e.g., slight differences in floating-point calculations) between these methods can lead to small discrepancies in the coefficients, especially in \(d\), which depends directly on the normal vector and the centroid.

---

### Which is "Correct"?
- **SVD Method**:
  - Provides a mathematically rigorous solution for minimizing orthogonal distances and is generally preferred for least-squares plane fitting in 3D.

- **Projection Method**:
  - Can be useful if you are specifically interested in the plane along a particular projection (e.g., minimizing errors in the \(z\)-axis for a top-down view).

The \(d\) value difference is a result of the fundamental difference in the plane-fitting criteria of the two methods. The SVD method treats all dimensions equally, while the determinant method does not.

# SVD-Based Method

**SVD-based method** for fitting a plane to a set of 3D points using equations and derivations in LaTeX and Markdown.

### **1. Problem Statement**

We want to fit a plane to a set of $n$ points in 3D space:
$$
\mathbf{P} = {(x_i, y_i, z_i)}_{i=1}^n
$$

The equation of the plane is:
$$
ax + by + cz + d = 0
$$
where $a$, $b$, and $c$ are the components of the plane's normal vector $\mathbf{n} = [a, b, c]$, and $d$ is the plane's offset from the origin.

---

### **2. Centering the Points**

To simplify calculations, we center the points by subtracting their centroid:
$$
\mathbf{p}_\text{centroid} = \left( \bar{x}, \bar{y}, \bar{z} \right)
$$
where:
$$
\bar{x} = \frac{1}{n} \sum_{i=1}^n x_i, \quad
\bar{y} = \frac{1}{n} \sum_{i=1}^n y_i, \quad
\bar{z} = \frac{1}{n} \sum_{i=1}^n z_i
$$

The centered points are:
$$
\mathbf{P}_c = \{(x_i - \bar{x}, y_i - \bar{y}, z_i - \bar{z})\}_{i=1}^n
$$

---

### **3. Matrix Form**

We represent the centered points as an $n \times 3$ matrix:
$$
\mathbf{X}_c = 
\begin{bmatrix}
x_1 - \bar{x} & y_1 - \bar{y} & z_1 - \bar{z} \\
x_2 - \bar{x} & y_2 - \bar{y} & z_2 - \bar{z} \\
\vdots & \vdots & \vdots \\
x_n - \bar{x} & y_n - \bar{y} & z_n - \bar{z}
\end{bmatrix}
$$

---

### **4. Singular Value Decomposition (SVD)**

The goal is to find the best-fitting plane that minimizes the orthogonal distances from the points to the plane. This is equivalent to finding the direction of smallest variance in the data.

Using SVD, we decompose $\mathbf{X}_c$:
$$
\mathbf{X}_c = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T
$$

where:

- $\mathbf{U}$ is an $n \times n$ orthogonal matrix,
- $\mathbf{\Sigma}$ is a diagonal matrix of singular values ($\sigma_1 \geq \sigma_2 \geq \sigma_3$),
- $\mathbf{V}$ is a $3 \times 3$ orthogonal matrix.

The rows of $\mathbf{V}$ (or equivalently, the columns of $\mathbf{V}^T$) are the principal axes of the data. The last row of $\mathbf{V}$, corresponding to the smallest singular value $\sigma_3$, is the normal vector of the plane.

---

### **5. Plane Normal**

Let the last row of \(\mathbf{V}\) be:
$$
\mathbf{n} = [a, b, c]
$$

Then, the plane's normal vector is:
$$
\mathbf{n} = \mathbf{v}_3
$$

---

### **6. Plane Offset ($d$)**

To compute $(d)$, we use the centroid $(\bar{x}, \bar{y}, \bar{z})$ and the normal vector $\mathbf{n}$:
$$
d = -\mathbf{n} \cdot \mathbf{p}_\text{centroid}
$$

This ensures the plane passes through the centroid of the data.

---

### **7. Final Plane Equation**

The equation of the best-fitting plane is:

$$
ax + by + cz + d = 0
$$

---

### **8. Python Implementation**

This math is implemented as follows:

```python
def best_fit_plane(points: np.ndarray) -> Plane:
    """
    Calculate the best-fit plane using Singular Value Decomposition (SVD).

    Args:
        points (np.ndarray): A 2D array of shape (n, 3), where each row is a 3D point.

    Returns:
        Plane: A Plane object with the normal vector, a point on the plane (centroid),
               and the offset from the origin.
    """
    # Compute the centroid of the points
    centroid = points.mean(axis=0)

    # Center the points
    centered_points = points - centroid

    # Perform Singular Value Decomposition
    _, _, vh = np.linalg.svd(centered_points)

    # The normal vector is the last row of V^T (or last column of V)
    normal_vector = vh[-1]

    # Compute the offset (d) using the centroid
    d = -np.dot(normal_vector, centroid)

    # Return the plane
    return Plane(normal=Vector3(normal_vector), point=Point3(centroid), d=d)
```